In [ ]:
## extract classes 

import os
import shutil

def extract_classes(source, classes):

    dirname = os.path.dirname(__file__)
    source_directory = os.path.join(dirname, source)
    print("source: " + source_directory)
  
    for c in classes:
        temp = os.path.join(dirname, c)
        print( c + ": " + temp)
        os.makedirs(temp, exist_ok=True)

    for filename in os.listdir(source_directory):
        for c in classes:
            if c in filename:
                shutil.move(os.path.join(source_directory, filename), os.path.join(dirname,c, filename))



extract_classes("raw" , ["cat" , "dog"])

In [ ]:
## split folders
# pip install split-folders

import splitfolders
import os

dirname = os.path.dirname(__file__)
# input: folder that contsins all classes sub folders
splitfolders.ratio(os.path.join(dirname, "raw"), output=os.path.join(dirname, "processed"), seed=42, ratio=(0.6, 0.2, 0.2))


In [16]:
## read and resized images
# pip install opencv-python

import glob
import cv2
import numpy as np
import os

target_size = (128, 128)

def read_resize_images(source_dir , categories):
    x_temp = []
    y_temp = []
    for category in categories:
        for filepath in glob.glob(os.path.join(source_dir, category, "*")):
            label = 0 if category == "cats" else 1
            img = cv2.imread(filepath)
            img = cv2.resize(img, target_size)
            img = np.array(img)
            y_temp.append(label)
            x_temp.append(img)

    print("folder: " + os.path.join(source_dir))
    print(len(x_temp))
    print(type(x_train)) 
    print(x_temp[0])   
    return x_temp , y_temp

# source file, catefories list
x_train , y_train = read_resize_images( "D:\\Artificial Intelligence\\Machine Learning\\_projects\\dogs_cats_classification\\data\\processed\\train", ["cat", "dog"])
x_test , y_test = read_resize_images( "D:\\Artificial Intelligence\\Machine Learning\\_projects\\dogs_cats_classification\\data\\processed\\test", ["cat", "dog"])
x_val , y_val = read_resize_images( "D:\\Artificial Intelligence\\Machine Learning\\_projects\\dogs_cats_classification\\data\\processed\\val", ["cat", "dog"])



folder: D:\Artificial Intelligence\Machine Learning\_projects\dogs_cats_classification\data\processed\train
15000
<class 'list'>
[[[ 87 164 203]
  [ 93 170 209]
  [ 94 171 210]
  ...
  [125 207 247]
  [119 204 245]
  [122 201 240]]

 [[ 87 164 203]
  [ 93 170 209]
  [ 94 171 210]
  ...
  [128 209 247]
  [122 205 245]
  [123 202 241]]

 [[ 87 164 203]
  [ 93 170 209]
  [ 94 171 210]
  ...
  [130 210 247]
  [125 206 244]
  [124 203 242]]

 ...

 [[ 53 124 158]
  [ 55 126 160]
  [ 56 127 161]
  ...
  [  0   4   3]
  [  0   4   3]
  [  0   2   2]]

 [[ 56 123 154]
  [ 58 125 156]
  [ 60 127 158]
  ...
  [  1   3   3]
  [  1   3   3]
  [  1   3   3]]

 [[ 54 121 152]
  [ 55 122 153]
  [ 59 126 157]
  ...
  [  0   2   2]
  [  0   2   2]
  [  0   2   2]]]
folder: D:\Artificial Intelligence\Machine Learning\_projects\dogs_cats_classification\data\processed\test
5000
<class 'list'>
[[[ 68 120 144]
  [ 62 114 137]
  [ 64 116 139]
  ...
  [179 155 209]
  [188 164 218]
  [177 159 208]]

 [[ 63 115

In [17]:
print(type(x_train[0]))

<class 'numpy.ndarray'>


In [23]:
## build model
# pip install tensorflow
# pip install keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    # Conv2D(64, (3, 3), activation='relu'),
    # MaxPooling2D((2, 2)),
    # Conv2D(128, (3, 3), activation='relu'),
    # MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification: cat or dog
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 63504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │     4,064,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,064,833 (15.51 MB)

 Trainable params: 4,064,833 (15.51 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:

# Train the model
history = model.fit(np.array(x_train), np.array(y_train), epochs=5, batch_size=32)
print(history)


Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 28s 50ms/step - accuracy: 0.9955 - loss: 0.1907
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 50ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 49ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 49ms/step - accuracy: 1.0000 - loss: 0.0000e+00


In [25]:
# Evaluate model
loss, accuracy = model.evaluate(np.array(x_test), np.array(y_test))
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Loss: 0.0, Test Accuracy: 1.0


In [28]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Predict on test data
y_pred = model.predict(np.array(x_test))

y_pred = (y_pred > 0.5).astype(int)
# Calculate precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)

# Build confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Precision: 1.0
Recall: 1.0
Confusion Matrix:
[[5000]]


c:\Users\MAHMOUD TAHA\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
# pip install pydot

from keras.utils import plot_model

# Plot the model architecture
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [33]:
## save the model with timestamp namd

import os
from datetime import datetime

# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Get the current working directory of the notebook
cwd = os.getcwd()

# Define the model name with the timestamp
model_name = f"model_{timestamp}.h5"

# Save the model
model.save(os.path.join(cwd, model_name))

# Print confirmation message
print("Model saved at:", os.path.join(cwd, model_name))


Model saved at: d:\Artificial Intelligence\Machine Learning\_projects\dogs_cats_classification\_notebooks\model_20240409_212734.h5


In [6]:
## load the model and predict random sample

import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tkinter import filedialog
import tkinter as tk
from datetime import datetime

# Function to preprocess the image
def preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize pixel values
    return img

# Function to open file dialog and choose an image
def choose_image():
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Show file dialog to choose an image
    file_path = filedialog.askopenfilename(title="Choose an image")

    return file_path

# Get the current working directory of the notebook
cwd = os.getcwd()

# Get all .h5 model files in the current directory
model_files = [file for file in os.listdir(cwd) if file.endswith('.h5')]

# Get the most recent model file
latest_model_file = max(model_files, key=os.path.getctime)

# Load the most recent model
model = load_model(os.path.join(cwd, latest_model_file))

# Call the function to choose an image
# img_path = choose_image()
img_path = "D:\\Artificial Intelligence\\Machine Learning\\_projects\\dogs_cats_classification\\data\\random\\random_cat.jpg"
if img_path:
    img = preprocess_image(img_path)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    if prediction[0][0] > 0.5:
        print("Prediction: Dog")
    else:
        print("Prediction: Cat")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
Prediction: Dog
